In this test file, we will use CMA-ES algorithm with Gaussian to do optimization for H2O molecule.

The purpose of this test is to check the availability of CMA-ES algorithm in molecule optimizaton.

In [1]:
# import necessary packages
import os
import re
import cma

In [2]:
# basic setting of calculations
nproc = "4"    # cpu cores used in calculations
mem = "4GB"    # maximum memory used in calculations
energy_keyword = "# B3LYP 6-31G*"    # the keyword of calculating energy,
charge = 0    # total charge of system
multiplicity = 1    # multiplicity of system
external_keyword = ""    # external keyword after the coordinates of atoms in Gaussian input files, like custom basis set setting
gaussian_commond = "g09"    # commond to call Gaussian calculation

In [3]:
# calculate the energy
def energy_calculate(atom_list, energy_keyword, charge=0, multiplicity=1, external_keyword=""):
    # write Gaussian gjf input file
    with open("energy.gjf", "wt") as txt:
        txt.write("%nproc=" + nproc + "\n")
        txt.write("%mem=" + mem + "\n")
        txt.write(energy_keyword + "\n\n")
        txt.write("energy_calculation\n\n")
        txt.write(str(charge) + " " + str(multiplicity) + "\n")
        for atom in atom_list:
            txt.write(atom + "\n")
        txt.write("\n"+external_keyword+"\n\n\n\n")

    # call Gaussian to calculate the energy
    try:
        os.system(gaussian_commond + " < energy.gjf > energy.log")
    except:
        print("Energy calculation failed. Check the energy.log file please!")

In [4]:
# extract the energy from Guassian output file
def energy_extract(energy_lable="HF=", filename="energy.log"):    # remeber to change energy_lable if using other calculation methods like post-HF or CC
    with open(filename, "rt") as txt:
        s = txt.read().replace("\n", "")
        s_list = s.split("\\")
        for i in range(len(s_list)-1):
            str = s_list[len(s_list)-1-i]
            if re.match(energy_lable, str) != None:
                return float(str[len(energy_lable):].replace(" ",""))

In [5]:
# define energy function for a specific system
def energy(coord):
    atom_list = ["O", "H 1 "+str(coord[0]), "H 1 "+str(coord[0])+" 2 "+str(coord[1])]    # use z-martix as an example and keep two O-H bonds the same
    energy_calculate(atom_list, energy_keyword, charge, multiplicity, external_keyword)
    return energy_extract()

In [6]:
# cma-es part
init_coord = [0.9584, 104.45]    # initial coordinates of atoms

es = cma.CMAEvolutionStrategy(init_coord, 0.2, {"seed":2445})    # set cma-es model, use 0.2 as sigma0, 2445 as random seed
es.optimize(energy)    # optimize the cma-es model

(3_w,6)-aCMA-ES (mu_w=2.0,w_1=63%) in dimension 2 (seed=2445, Tue Oct 11 17:16:16 2022)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1      6 -7.640654990000000e+01 1.0e+00 1.53e-01  1e-01  1e-01 0:04.1
    2     12 -7.640884090000000e+01 1.0e+00 1.39e-01  1e-01  1e-01 0:07.8
    3     18 -7.640750740000000e+01 1.2e+00 1.09e-01  7e-02  9e-02 0:11.6
    4     24 -7.640856900000000e+01 1.3e+00 9.35e-02  5e-02  7e-02 0:15.3
    6     36 -7.640853300000001e+01 1.5e+00 6.59e-02  3e-02  5e-02 0:22.7
    8     48 -7.640893410000000e+01 2.4e+00 4.39e-02  1e-02  3e-02 0:30.0
   10     60 -7.640888090000000e+01 2.3e+00 3.86e-02  9e-03  2e-02 0:37.4
   12     72 -7.640893749999999e+01 2.5e+00 3.20e-02  6e-03  2e-02 0:44.8
   15     90 -7.640893840000000e+01 6.0e+00 3.25e-02  4e-03  2e-02 0:55.9
   18    108 -7.640892839999999e+01 9.2e+00 3.07e-02  2e-03  2e-02 1:06.9
   21    126 -7.640893810000000e+01 1.1e+01 2.08e-02  1e-03  1e-02 1:18.0
   24    144 -7.6408939899

In [7]:
print(es.result.xbest)    # print optimization result

[  0.96878134 103.64849749]


The optimization result is:

O-H bond is 0.96878134 A;

H-O-H angle is 103.64849749 degree.

****************

If we directly use Gaussian to do optimization with "opt=tight", we will get the result:

O-H bond is 0.9687 A;

H-O-H angle is 103.6522 degree.

****************

The result is almost the same, showing using CMA-ES algorithm to do optimization for molecules is available.